In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [ ]:
plt.style.use('dark_background') 

def highlight_odd_rows(s):
    styles = []
    for i in range(len(s)):
        if i % 2 == 1:
            styles.append('background-color: indigo; border: 1px solid white;')
        else:
            styles.append('border: 1px solid white;')
    return styles

In [ ]:
df = pd.read_csv("consultassec.csv")

In [ ]:
df.head().style.apply(highlight_odd_rows)

In [ ]:
def getDfSize(df):
    rows = len(df.axes[0])
    columns = len(df.axes[1])

    return {'rows': rows, 'columns': columns}

In [ ]:
getDfSize(df)

<h3>La siguiente función extrae las distintas llaves de cada diccionario que contenga una columna como valor en el dataframe</h3>

In [ ]:
def determineDistinctKeys(columnToList): #Recibe una lista, no una dataframe column
    distinctKeysDict = {}
    distinctKeysList = []
    for dictionary in columnToList:
        dictionary = json.loads(dictionary)
        orderedKeys = sorted(list(dictionary.keys()))
        if orderedKeys in distinctKeysList:
            distinctKeysDict['/'.join(orderedKeys)] += 1
        else:
            distinctKeysList.append(orderedKeys)
            distinctKeysDict['/'.join(orderedKeys)] = 0
    return distinctKeysDict

<h3>La siguiente función extrae los distintos valores de una columna en el dataframe</h3>

In [ ]:
def determineDistinctValues(df, col): #Recibe el dataframe y la columna del dataframe que requiere ser revisado
    return df[col].unique()

<h3>Con esto sabemos que la columna "Response Body" tiene un cuerpo idéntico para todos los casos</h3>

In [ ]:
responseBody = df['responseBody'].values.tolist()

In [ ]:
determineDistinctKeys(responseBody)

<h3>Con esto sabemos la proporción entre consultas con algún tipo de error, o consultas normales en Carfax  USA</h3>

In [ ]:
carfaxUsaData = df['carfaxUsaData'].values.tolist()

In [ ]:
determineDistinctKeys(carfaxUsaData)

<h3>Finalmente sabemos qué se pide en los "Request Parameters"</h3>

In [ ]:
requestParameters = df['requestParameters'].values.tolist()

In [ ]:
determineDistinctKeys(requestParameters)

<h3>Ahora nos aseguramos de todos los valores distintos que haya en las columnas</h3>

In [ ]:
determineDistinctValues(df, 'apiKey') #Solo existe un valor posible para este campo

In [ ]:
determineDistinctValues(df, 'userId') #Solo existe un valor posible para este campo

In [ ]:
determineDistinctValues(df, 'idReporte') #Existen varios valores (¿de qué depende esta diferencia?)

In [ ]:
determineDistinctValues(df, 'responseCode') #Solo existe un valor posible para este campo

In [ ]:
determineDistinctValues(df, 'responseCodeStatus') #Solo existe un valor posible para este campo

In [ ]:
determineDistinctValues(df, 'labels') #Solo existe un valor posible para este campo

In [ ]:
determineDistinctValues(df, 'firewallUsa') #Solo existe un valor posible para este campo

<h3>A continuación busco los distintos tipos de código de alerta que se encuentran en el dataframe creando una función especial para ello </h3>

In [ ]:
def countAlertCodes(df, alertsColumn: str):
    alertsWithCode = list(determineDistinctValues(df, alertsColumn))
    codeCount = {'Code 1': 0 , 'Code 2': 0, 'Code 3': 0, 'Code 4': 0, 'Code 5': 0, 'Code 6': 0, 'Code 7': 0}

    for alert in alertsWithCode:
        alertList = json.loads(alert)
        if alertList != []:
            for subalert in alertList:
                n = subalert['codigo']
                codeCount[''.join(['Code ', str(n)])] += 1

    return codeCount #El resultado me dará el código de error más frecuente en la plataforma


In [ ]:
countAlertCodes(df, 'alertas')

<h3>Ahora calculo el tiempo en que se tarda cada petición en generar una respuesta</h3>

In [ ]:
def calculateDeltaTime(df, columnA, columnB):
    responseTimeType = pd.to_datetime(df[columnA])
    requestTimeType = pd.to_datetime(df[columnB])
    responsePeriod = (responseTimeType - requestTimeType).to_frame()
    responsePeriod['secondsDifference'] = pd.to_timedelta(responsePeriod[0]).dt.total_seconds()
    responsePeriod.drop([0], axis='columns', inplace=True)

    return responsePeriod

<h4>Con mayor frecuencia parece tardarse un segundo la respuesta. La gráfica parece ajustarse a una distribución de Poisson o a una distribución normal. Checar cuál es la más conveniente para calcular probabilidades</h4>

In [ ]:
plt.hist(calculateDeltaTime(df, 'responseTime', 'requestTime')['secondsDifference'], bins = 20, edgecolor = 'black')
plt.xlabel('Time in Seconds')
plt.ylabel('Frequency')
plt.title('Histogram of Time in Seconds')
plt.show()

<h3>A continuación se comprueba que cada diccionario del valor de la columna carfaxUsaData con una llave "error" le corresponda un valor distinto a lista vacía</h3>

In [ ]:
def proveErrorAlert(df):
    carfaxUsaData = df['carfaxUsaData'].values.tolist()
    conError = 0
    for index1 in range(len(carfaxUsaData)):
        carfaxDict = json.loads(carfaxUsaData[index1])
        if 'error' in list(carfaxDict.keys()):
            conError += 1
            if df.iloc[index1]['alertas'] == []:
                return 'Existe un valor con error y sin alerta'
    return 'Funciona correctamente'

In [ ]:
proveErrorAlert(df)

<h3>Ahora se estudiará la relación con los Vin's repetidos y las alertas</h3>

In [ ]:
#The next function identifies the repeated values of a column

def getRepeatedValuesInAColumn(df, columnName):
    duplicates = df[df.duplicated(subset=[columnName], keep=False)]
    return duplicates['VIN']

In [ ]:
def searchValuesInADataframe(values:list, columnA: str, columnB: str, df)-> dict:
    '''
    Searches the values of a list in a dataframe columnA and retrieves the information of columnB
    Args:
       values (list): The list of values to be searched
       columnA (string): The name's column to search
       columnB (string): The name's column to retrieve values
       df (pandas dataFrame): The dataframe where where will be searched
    Returns:
      valuesBDict (dict): A dictionary of values as key and the retrieved values of columnB as values
    '''
    
    valuesBDict ={}
    for value in values:
        valuesB = df.loc[df[columnA]== value, columnB].tolist()
        valuesBDict[value] = valuesB
    return valuesBDict

<h4>Identificamos si existe algún VIN repetido que contenga y a la vez no, mensajes de alerta. Además se identifica la cantidad de veces que se repiten los VIN's que lanzan alerta contra los que no lanzan ninguna alerta</h4>

In [ ]:
def getIncorrectRepeatedVinInformation(df):
    repeatedRows = len(getRepeatedValuesInAColumn(df, 'VIN').values.tolist())
    vinValues = getRepeatedValuesInAColumn(df, 'VIN').unique().tolist()
    repeatedVins = len(vinValues)
    repeatedValues = searchValuesInADataframe(vinValues, 'VIN', 'alertas', df)
    goodVins = 0
    badVins = 0
    goodVinsList = []
    badVinsList = []
    for repeatedVin in list(repeatedValues.keys()):
        differentValuesByVin = list(set(repeatedValues[repeatedVin]))
        for differentValue in differentValuesByVin:
            if differentValue == '[]':
                goodVins += 1
                if len(differentValuesByVin)>1:
                    return {message: ''.join(['Alerta con vin: ',repeatedVin, ' , tiene y no tiene alertas.']),
                           'goodVinsList': [],
                           'badVinsList': [],
                           'goodVins':0,
                           'badVins': 0}
                else:
                    goodVinsList.append([repeatedVin, len(repeatedValues[repeatedVin])])
            else:
                badVins += 1
                badVinsList.append([repeatedVin, len(repeatedValues[repeatedVin])])
            

    return {'message': ''.join(['VINs repetidos que no lanzan alerta: ', str(goodVins), '\nVINs repetidos que sí lanzan alerta: ', str(badVins)]),
           'goodVinsList': goodVinsList,
           'badVinsList': badVinsList,
           'goodVins':goodVins,
           'badVins': badVins}

In [ ]:
def diagnoseVins(df):
    goodVinsFrame = pd.DataFrame(getIncorrectRepeatedVinInformation(df)['goodVinsList'], columns = ['VIN', 'repeatedTimes'])
    goodRepeatedRecords = goodVinsFrame.sum()['repeatedTimes']
    badVinsFrame = pd.DataFrame(getIncorrectRepeatedVinInformation(df)['badVinsList'], columns = ['VIN', 'repeatedTimes'])
    badRepeatedRecords = badVinsFrame.sum()['repeatedTimes']

    return {'goodVinsFrame': goodVinsFrame, 'badVinsFrame': badVinsFrame, 'goodRepeatedRecords': goodRepeatedRecords, 'badRepeatedRecords': badRepeatedRecords}

In [ ]:
diagnoseVins(df)['goodVinsFrame'].describe().style.apply(highlight_odd_rows)

In [ ]:
plt.hist(diagnoseVins(df)['goodVinsFrame']['repeatedTimes'], bins = 50, edgecolor = 'black')
plt.xlabel('Repeated Times')
plt.ylabel('Frequency')
plt.title('Histogram of Repeated Times by VIN (without alerts)')
plt.show()

In [ ]:
diagnoseVins(df)['badVinsFrame'].describe().style.apply(highlight_odd_rows)

In [ ]:
plt.hist(diagnoseVins(df)['goodVinsFrame']['repeatedTimes'], bins =10, edgecolor = 'black')
plt.xlabel('Repeated Times')
plt.ylabel('Frequency')
plt.title('Histogram of Repeated Times by VIN (with alerts)')
plt.show()

<h4>Finalmente tenemos los VINs revisados en todos los registros. La proporción entre VINs repetidos con alerta, VINs repetidos sin alerta. Máximos y mínimos repeticiones</h4>

In [ ]:
print('Cantidad de registros repetidos: ', diagnoseVins(df)['goodRepeatedRecords'] + diagnoseVins(df)['badRepeatedRecords'])
print('Cantidad de registros repetidos con VINs sin alerta: ', diagnoseVins(df)['goodRepeatedRecords'])
print('Cantidad de registros repetidos con VINs con alerta: ', diagnoseVins(df)['badRepeatedRecords'])
print('Cantidad de VINs revisados en esos registros repetidos: ', list(diagnoseVins(df)['badVinsFrame'].count() + diagnoseVins(df)['goodVinsFrame'].count())[1])
repetitionProportion = [diagnoseVins(df)['goodRepeatedRecords'], diagnoseVins(df)['badRepeatedRecords']]
labelsRepetition = ['VINs without alerts', 'VINs with alerts']
plt.pie(repetitionProportion, labels=labelsRepetition, autopct="%0.1f %%")
plt.axis("equal")
plt.title('Repeated records')
plt.show()

<h4>Estadísticas de las repeticiones de los VINs sin alerta</h4>

In [ ]:
diagnoseVins(df)['goodVinsFrame'].describe().style.apply(highlight_odd_rows)

<h4>Estadísticas de las repeticiones de los VINs con alerta</h4>

In [ ]:
diagnoseVins(df)['badVinsFrame'].describe().style.apply(highlight_odd_rows)

<h3>Ahora se comprueba que para cada VIN repetido y sin alerta se dió exactamente la misma información</h3>

In [ ]:
def verifyInfoRepeatedVins(df):
    uniqueResponses = {}
    differentResponses = {}
    vinValues = getRepeatedValuesInAColumn(df, 'VIN').unique().tolist()
    repeatedValues = searchValuesInADataframe(vinValues, 'VIN', 'responseBody', df)
    repeatedVins = repeatedValues.keys()
    for vin in repeatedVins:
        uniqueResponses[vin] = []
        listJson = []
        for responseString in repeatedValues[vin]:
            responseJson = json.loads(responseString)
            listJson.append(responseJson)
        repeatedValues[vin] = listJson
    for vin in repeatedVins:
        for responseJson in repeatedValues[vin]:
            info = {'anioModelo': responseJson['anioModelo'], 'fabricante': responseJson['fabricante'], 'marca': responseJson['marca'], 'modelo': responseJson['modelo'], 'paisOrigen': responseJson['paisOrigen'], 'robo': responseJson['robo'], 'roboFecha': responseJson['roboFecha']}
            if uniqueResponses[vin] == []:
                 uniqueResponses[vin].append(info)
            else:
                if info not in uniqueResponses[vin]:
                    uniqueResponses[vin].append(info)
                    info['tiempoRespuesta'] = responseJson['tiempoRespuesta']
                    differentResponses[vin] = info
    return {'uniqueResponses': uniqueResponses, 'differentResponses': differentResponses}

In [ ]:
if verifyInfoRepeatedVins(df)['differentResponses'] != {}:
    print('Existen respuestas distintas para un mismo VIN')

<h3>Ahora se comprueba que para cada VIN repetido y con alerta se dió exactamente la misma información y los mismos códigos de alerta</h3>

<h3>Ahora se analizan una parte de los datos mediante un periodo de tiempo n variable que extrae un subconjunto del dataframe </h3>

<h4>Primero se ordenan los datos de manera descendente por el campo "responseTime"</h4>

In [ ]:
df = df.sort_values(by = 'responseTime', ascending = False)

In [ ]:
df['responseTime'] = pd.to_datetime(df['responseTime'], format='%d/%m/%Y %H:%M:%S')
n_minutes  = 10
first_timestamp = df['responseTime'].iloc[0]
time_threshold = first_timestamp - pd.Timedelta(minutes=n_minutes)
filtered_df = df[df['responseTime'] >= time_threshold]
print(filtered_df.shape)